### Install Requirements

In [1]:
!pip install -q accelerate==1.3.0 peft==0.14.0 bitsandbytes==0.45.2 transformers==4.48.3 trl==0.14.0 pretty_midi

In [2]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [10]:
# Load the training dataset
from google.colab import drive
import pandas as pd

df = pd.read_csv('/content/imdb/train_data.txt', delimiter=":::", names=["index", "title", "genre", "description"])

<ipython-input-10-dbc0b2345c43>:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('/content/imdb/train_data.txt', delimiter=":::", names=["index", "title", "genre", "description"])


In [13]:
dataset_df = pd.DataFrame()
def form_prompt(data):
  return f"### Instruction:\n Given a description about a movie, analyse the given description of the movie and return the genre of the movie.\n### Description:\n{data}"

def form_completion(data):
  return f"### Genre:\n{data}"

dataset_df["prompt"] = df["title"].apply(form_prompt)
dataset_df["completion"] = df["genre"].apply(form_completion)

In [14]:
from datasets import Dataset
dataset = Dataset.from_pandas(dataset_df)

In [15]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)